# Nama : Alya Fauziyah
# NIM : 21110030
# Kelas : SD02A

In [1]:
# Import Library
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


from string import punctuation
import re

# Text-Preprocessing

In [2]:
# Membaca dataset
df=pd.read_csv("/content/dataset_mentalhealth.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  98 non-null     int64 
 1   Questions    98 non-null     object
 2   Jawaban      98 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [3]:
# Menampilkan lima baris pertama dari dataset
df.head()

,Question_ID,Questions,Jawaban
0,1590140,Apa yang dimaksud dengan penyakit mental?,Penyakit mental adalah kondisi kesehatan yang ...
1,2110618,Siapa yang terpengaruh oleh penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
2,6361820,Apa penyebab penyakit mental?,Diperkirakan bahwa penyakit mental mempengaruh...
3,9434130,Apa sajakah tanda-tanda peringatan penyakit me...,Gejala gangguan kesehatan mental bervariasi te...
4,7657263,Apakah penderita penyakit jiwa bisa sembuh?,"Ketika penyembuhan dari penyakit mental, ident..."


In [4]:
# Memeriksa adanya nilai kosong dalam dataset
df.isnull()

,Question_ID,Questions,Jawaban
0,False,False,False
1,False,False,False
2,False,False,False
3,False,False,False
4,False,False,False
...,...,...,...
93,False,False,False
94,False,False,False
95,False,False,False
96,False,False,False


In [5]:
# Pra-pemrosesan data
# Mengonversi teks menjadi huruf kecil
df['Questions'] = df['Questions'].str.lower()

In [6]:
# Menghilangkan karakter yang tidak perlu
df['Questions'] = df['Questions'].str.replace('[^\w\s]', '')

In [7]:
# Menangani nilai yang hilang
df.dropna(inplace=True)

# Normalisasi/ vektorisasi

In [8]:
# Menghitung frekuensi intent untuk visualisasi
intent_counts = df['Questions'].value_counts()

In [9]:
# Visualisasi distribusi intent
fig = px.bar(x=intent_counts.index, y=intent_counts.values, labels={'x': 'Intents', 'y': 'Count'})
fig.show()

In [10]:
# Menyalin kolom 'Questions' ke 'Intent' untuk menghitung jumlah pertanyaan
df['Intent'] = df['Questions']
questions_response_counts = df.groupby('Intent').size().reset_index(name='Count')
questions_response_counts

,Intent,Count
0,apa buktinya tentang vaping,1
1,apa itu distimia atau gangguan depresi persisten,1
2,apa itu gangguan kepribadian antisosial,1
3,apa itu gangguan kepribadian obsesifkompulsif,1
4,apa itu gangguan kepribadian skizoid,1
...,...,...
92,saya sudah dewasa dan saya pikir saya mungkin ...,1
93,seseorang yang saya cintai telah didiagnosis m...,1
94,seseorang yang saya cintai telah didiagnosis m...,1
95,seseorang yang saya sayangi mengalami cedera o...,1


In [11]:
# Menghitung rata-rata jumlah pertanyaan per intent
avg_questions = df.groupby('Intent').size().mean()

# Visualisasi rata-rata jumlah pertanyaan per intent
fig = px.bar(x=questions_response_counts['Intent'], y=questions_response_counts['Count'],
             labels={'x': 'Intents', 'y': 'Average Questions Count'})
fig.show()

# Modeling

In [12]:
# Membagi dataset menjadi data training dan testing
X_train, X_test, y_train, y_test = train_test_split(df['Questions'], df['Intent'], test_size=0.2, random_state=42)

In [13]:
# Melakukan vektorisasi pada data teks
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train/Melatih model
model = LinearSVC()
model.fit(X_train_vec, y_train)


LinearSVC()

# Evaluasi model

In [14]:
# Menilai model menggunakan metrik klasifikasi
y_pred = model.predict(X_test_vec)
report = classification_report(y_test, y_pred)
print(report)

                                                                                                                                                                                                                                                                                                            precision    recall  f1-score   support

                                                                                                                                                                                                                                                                   apa itu gangguan kepribadian antisosial       0.00      0.00      0.00       1.0
                                                                                                                                                                                                                                                                      apa itu gangguan kepribadian skizoid       0.00      0.00

In [16]:
# Visualisasi model performance
metrics = ['precision', 'recall', 'f1-score', 'support']
scores = classification_report(y_test, y_pred, output_dict=True)['weighted avg']

fig = px.bar(x=metrics, y=[scores[metric] for metric in metrics], labels={'x': 'Metrics', 'y': 'Score'})
fig.show()

# Uji/ testing dengan 10 pertanyaan

In [19]:
import csv

# Inisialisasi list untuk menyimpan data
chatbot_responses = []

print("Selamat Datang di Mental Health FAQ Chatbot!")
print("Ajukan pertanyaan atau enter, 'quit' untuk keluar.")

while True:
    user_input = input("Masukkan pertanyaan: ")

    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    # Melakukan vektorisasi pada input pengguna
    user_input_vec = vectorizer.transform([user_input.lower()])

    # Memprediksi intent
    predicted_intent = model.predict(user_input_vec)[0]

    # Menerapkan mekanisme generasi respons berdasarkan intent yang diprediksi
    response = df[df['Questions'] == predicted_intent]['Jawaban'].values[0] if predicted_intent in df['Questions'].values else "Maaf, saya tidak punya jawaban untuk pertanyaan itu."

    # Menampilkan respons
    print("Chatbot: " + response)

    # Menambahkan pertanyaan dan respons ke list
    chatbot_responses.append({"Pertanyaan": user_input, "Jawaban": response})

# Menyimpan hasil ke dalam file CSV
csv_filename = "hasil_chatbot_responses.csv"
fields = ["Pertanyaan", "Jawaban"]

with open(csv_filename, mode='w', newline='') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fields)

    # Menulis header
    writer.writeheader()

    # Menulis data
    for row in chatbot_responses:
        writer.writerow(row)

print(f"Hasil chatbot telah disimpan dalam file {csv_filename}.")

Selamat Datang di Mental Health FAQ Chatbot!
Ajukan pertanyaan atau enter, 'quit' untuk keluar.
Masukkan pertanyaan: Berapa banyak orang dewasa di Amerika yang diperkirakan terpengaruh oleh penyakit mental?
Chatbot: Diperkirakan bahwa penyakit mental mempengaruhi 1 dari 5 orang dewasa di Amerika, dan bahwa 1 dari 24 orang dewasa memiliki penyakit mental yang serius. Penyakit mental tidak membeda -bedakan; Ini dapat mempengaruhi siapa pun, terlepas dari jenis kelamin, usia, pendapatan, status sosial, etnis, agama, orientasi seksual, atau latar belakang.
Meskipun penyakit mental dapat mempengaruhi siapa pun, kondisi tertentu mungkin lebih umum pada populasi yang berbeda. Misalnya, gangguan makan cenderung terjadi lebih sering pada wanita, sementara gangguan seperti defisit perhatian/gangguan hiperaktif lebih lazim pada anak -anak.
Selain itu, segala usia rentan, tetapi yang muda dan yang tua sangat rentan. Penyakit mental biasanya mengejutkan individu dalam hidup mereka, dengan 75 persen

# Analisis

Untuk menganalisis hasil uji/tes dengan 10 pertanyaan di atas,  dapat menilai sejauh mana chatbot memberikan jawaban yang tepat atau relevan. Berikut adalah hasil analisis:

1. Pertanyaan: Berapa banyak orang dewasa di Amerika yang diperkirakan terpengaruh oleh penyakit mental?
* Jawaban: Chatbot memberikan jawaban yang sangat relevan dan informatif tentang prevalensi penyakit mental di Amerika. Jawaban tepat dan relevan.



2. Pertanyaan: Apa pengertian gangguan kepribadian?
* Jawaban: Chatbot memberikan definisi yang jelas dan informatif tentang gangguan kepribadian. Jawaban cukup tepat dan memberikan pemahaman dasar.


3. Pertanyaan: Sebutkan perbedaan mendasar antara kecemasan dan stres?
* Jawaban: Chatbot memberikan penjelasan yang membedakan antara kecemasan dan stres. Jawaban sangat tepat dan informatif.


4. Pertanyaan: Di mana saya bisa menemukan informasi terkait uji klinis?
* Jawaban: Chatbot memberikan informasi yang berguna dan memberikan arahan yang sesuai. Jawaban tepat dan relevan.


5. Pertanyaan: Apa ciri-ciri yang mencolok dari gangguan kepribadian skizoid?
* Jawaban: Chatbot memberikan penjelasan yang baik tentang ciri-ciri gangguan kepribadian skizoid. Jawaban sangat tepat dan informatif.


6. Pertanyaan: Bagaimana pendapat Anda tentang mencari pendapat independen dari seorang psikiater lain pada saat ini?
* Jawaban: Chatbot memberikan panduan yang sesuai tentang mencari pendapat independen dari psikiater lain. Jawaban cukup tepat, memberikan informasi yang berguna.

7. Pertanyaan: Sebutkan beberapa jenis profesional kesehatan mental?
* Jawaban: Chatbot memberikan informasi yang memadai tentang berbagai jenis profesional kesehatan mental. Jawaban cukup tepat dan memberikan pemahaman umum.


8. Pertanyaan: Apa yang mungkin menjadi penyebab timbulnya masalah kesehatan mental?
* Jawaban: Chatbot memberikan jawaban yang mencakup beberapa faktor penyebab masalah kesehatan mental. Jawaban tepat dan memberikan wawasan yang baik.


9. Pertanyaan: Apakah ada jenis terapi medik yang efektif untuk menanggulangi kondisi kesehatan mental?
* Jawaban: Chatbot memberikan jawaban yang mencerminkan pendekatan holistik terhadap penanganan masalah kesehatan mental. Jawaban cukup tepat, memberikan pemahaman umum.


10. Pertanyaan: Apa saja tanda umum yang dapat dianggap sebagai ciri-ciri gangguan kesehatan mental pada orang dewasa?
* Jawaban: Chatbot memberikan jawaban yang memberikan pemahaman tentang tanda-tanda gangguan kesehatan mental pada orang dewasa. Jawaban sangat tepat dan informatif.



# Kesimpulan:
Dari 10 pertanyaan yang diajukan, jawaban yang diberikan oleh chatbot cenderung memberikan informasi yang relevan dan cukup tepat dalam sebagian besar kasus. Sebagai contoh, saat ditanya tentang jumlah orang dewasa yang terpengaruh oleh penyakit mental di Amerika, chatbot memberikan jawaban yang memberikan informasi umum dan relevan mengenai prevalensi penyakit mental. Selain itu, ketika pertanyaan berkaitan dengan pengertian gangguan kepribadian atau perbedaan antara kecemasan dan stres, chatbot memberikan jawaban yang jelas dan informatif.


<P>
Namun, perlu diingat bahwa pengujian ini tidak mencakup semua kemungkinan pertanyaan atau skenario, dan kualitas respons dapat dipengaruhi oleh keberagaman data yang digunakan untuk melatih model. Oleh karena itu, penting untuk terus meningkatkan dan memvalidasi model chatbot dengan dataset yang lebih besar dan beragam untuk meningkatkan kinerjanya. Meskipun demikian, secara keseluruhan, chatbot ini memberikan pengalaman interaksi yang cukup memadai dalam merespons pertanyaan seputar kesehatan mental.





